In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F
import pandas as pd

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/26 18:52:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.3.2'

In [9]:
schema = types.StructType([ 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
    ])

In [10]:
df = spark.read \
     .option("header","true") \
     .schema(schema) \
     .csv('fhvhv_tripdata_2021-06.csv')

In [12]:
df.repartition(12).write.parquet('fhvhv/2021/06/',mode='overwrite')

23/02/26 19:01:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/26 19:01:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/26 19:01:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/26 19:01:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/26 19:01:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/02/26 19:01:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/26 19:01:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/26 19:01:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/26 19:01:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [25]:
df.filter(df.pickup_datetime >= '2021-06-15 00:00:00') \
  .filter(df.pickup_datetime <= '2021-06-15 23:59:59').count()

452470

In [29]:
df.registerTempTable('trip_data')

/home/wtpoll/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [43]:
df0=spark.sql("""select date(pickup_datetime),
                 max(
                    bigint(to_timestamp(dropoff_datetime))-
                    bigint(to_timestamp(pickup_datetime)))/3600.0 
                 from trip_data
                 group by 1 order by 2 desc""").show()

+---------------+--------------------------------------------------------------------------------+
|pickup_datetime|(max((to_timestamp(dropoff_datetime) - to_timestamp(pickup_datetime))) / 3600.0)|
+---------------+--------------------------------------------------------------------------------+
|     2021-06-25|                                                                       66.878889|
|     2021-06-22|                                                                       25.549722|
|     2021-06-27|                                                                       19.980833|
|     2021-06-26|                                                                       18.197222|
|     2021-06-23|                                                                       16.466944|
|     2021-06-24|                                                                       13.909722|
|     2021-06-04|                                                                       11.670000|
|     2021

In [33]:
schema_zone=types.StructType([ 
    types.StructField('LocationID', types.IntegerType(), True), 
    types.StructField('Borough', types.StringType(), True), 
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
    ])

In [34]:
df_zone = spark.read \
     .option("header","true") \
     .schema(schema_zone) \
     .csv('taxi_zone_lookup.csv')

In [35]:
df_zone.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [36]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [37]:
df_zone.registerTempTable("zones")

/home/wtpoll/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [40]:
spark.sql("""select Zone,count(1) from 
                   trip_data join zones on PULocationID=LocationID 
                   group by 1 
                   order by 2 desc 
                   limit 1""").show()

+-------------------+--------+
|               Zone|count(1)|
+-------------------+--------+
|Crown Heights North|  231279|
+-------------------+--------+

